# Alcohol

In [1]:
import pandas as pd
import time
import numpy as np
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

df = pd.read_csv('preprocessed_collisions.csv')

/Users/Marie/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (7,8,9,22,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Make dataframe

In [173]:
df['AnyInjured'] = (df['NUMBER OF PERSONS INJURED'] > 0).astype(int)
df['AnyKilled']  = (df['NUMBER OF PERSONS KILLED'] > 0).astype(int)

df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S')

# Make dataframe
df_serious = df.loc[(df['NUMBER OF PERSONS KILLED'] + df['NUMBER OF PERSONS INJURED']) > 0]
df_serious['TIME'] = pd.to_datetime(df_serious['CRASH DATE'] + ' ' + df_serious['CRASH TIME']) #Takes approximately 1 minute.

# Discard 2012 and 2021.
df_serious = df_serious.loc[~((df_serious['TIME'].dt.year == 2012) | (df_serious['TIME'].dt.year == 2021))]

df_serious_alco =  df_serious[(df_serious['CF_Alcohol Involvement'] == 1)]
df_alco =  df[(df['CF_Alcohol Involvement'] == 1)]

# Add time
df_serious_alco['YEAR'] = df_serious_alco['TIME'].dt.year
df_serious_alco['MONTH'] = df_serious_alco['TIME'].dt.month
df_serious_alco['WEEKDAY'] = df_serious_alco['TIME'].dt.weekday
df_serious_alco['HOUR'] = df_serious_alco['TIME'].dt.hour

df_alco['YEAR'] = df_alco['TIME'].dt.year
df_alco['MONTH'] = df_alco['TIME'].dt.month
df_alco['WEEKDAY'] = df_alco['TIME'].dt.weekday
df_alco['HOUR'] = df_alco['TIME'].dt.hour

/Users/Marie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Marie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Marie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [ ]:
plot1 = pd.DataFrame(df_serious_alco.groupby('YEAR')['NUMBER OF PERSONS INJURED'].size().reset_index().values, columns=['Year','Number of collisions'])

# Total counts
df_temp11 = df_alco.groupby('YEAR')['NUMBER OF PERSONS INJURED'].sum().reset_index().astype(int); df_temp11.columns = ['Year','Total Count']

# Number of serious count 
df_temp22 = pd.DataFrame(df_alco.loc[(df_alco['AnyInjured']|df_alco['AnyKilled']).astype(bool)].groupby('YEAR').size().reset_index()); df_temp22.columns = ['Year', 'Serious Count']

# Merge 
df_temp33 = pd.merge(df_temp11, df_temp22, how='left')

# Add probability
df_temp33['Probability of being either injuring or deadly'] = (df_temp33['Serious Count']/df_temp33['Total Count'])*100

df_temp33['change'] = df_temp33['Probability of being either injuring or deadly'].pct_change().fillna(0)*100

### Plot

In [204]:
fig = make_subplots(rows=2, cols=1,
                   subplot_titles=("Seriousness of alcohol involment collisions",
                                   "Number of collisions where alcohol was a contributing factor"))

# Plot probability
fig.add_trace(
    go.Scatter(x = df_temp33["Year"], 
               y = df_temp33["Probability of being either injuring or deadly"],
               line_color='#2bae80',
               name = " "),
    row=1,col=1,
)
fig.update_traces(
    hovertemplate="<br>".join([
        "Year: %{x}",
        "Probability of collision being either injuring or deadly given",
        "that alcohol involment was a contributing factor: %{y:.2f} %"]
    ),
)

# Plot count
fig.add_trace(
    go.Scatter(x = plot1["Year"], y = plot1["Number of collisions"],
               line_color='#461969',
    name = " "),
    row=2,col=1  
) 


# Add text
fig.add_annotation(x=2016, y=65.8,
            text="Vision Zero launch" ,
            showarrow=False,
            yshift=10)

fig.add_annotation(x=2016, y=64.2,
            text="'Choices' campaign" ,
            showarrow=True,
            yshift=10)

# Update x,y-axis properties
fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Probability", row=1, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)

fig.update_layout(showlegend=False)

fig.show()

Diving into the contributing factor 'alcohol involment'. In the last plot we see a similar distribution as the one in the explainer notebook, which looked into the total number of collisions. However in the this plot we have restricted it to only count the ones, where alcohol involment has been a contributing factor of the collision. From that plot, we sort of concluded that we could not see any effect over the years of Vision Zero's campaign 'Choices', which taggets driving while influenced by alcohol directly by showing clear images of how wrong it can go. The first plot above shows on the other hand the seriousness of alcohol involment, i.e. given you are in a moter vehical collision, what is the probability of that collision being serious? Looking specifically at collisions that resultet in being serious, we do on the other hans see an effect. The campaign was launched in March 2016 and already in 2017 the change from the previous year was decreased with 2.23 %. The succes of the campaign was on the other hand short, and already in 2018 the probability increased with 1.24 %.

__[Link to Visions Zero campaign 'Choices'](http://www.nyc.gov/html/visionzero/pages/initiatives/choices.shtml)__